In [33]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [32]:
import pandas as pd
import xml.etree.ElementTree as ET
import psycopg2
from datetime import datetime
import re
import pytz
from psycopg2.extras import execute_batch

# Database connection parameters
db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "C:\\Users\\DELL\\OneDrive\\Documents\\GitHub\\TPRelex\\RELEX-Y\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "C:\\Users\\DELL\\OneDrive\\Documents\\GitHub\\TPRelex\\RELEX-Y\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "C:\\Users\\DELL\\OneDrive\\Documents\\GitHub\\TPRelex\\RELEX-Y\\CSPOMS_DEV_Conn\\server-ca.pem"
}

connection = psycopg2.connect(**db_params)
cursor = connection.cursor()
print('test')

test


In [2]:
pip install psycopg2-binary sqlalchemy --user


  Using cached psycopg2_binary-2.9.9-cp311-cp311-win_amd64.whl.metadata (4.6 kB)
Using cached psycopg2_binary-2.9.9-cp311-cp311-win_amd64.whl (1.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [29]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
import numpy as np
import urllib.parse

# Database connection parameters
db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "C:\\Users\\DELL\\OneDrive\\Documents\\GitHub\\TPRelex\\RELEX-Y\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "C:\\Users\\DELL\\OneDrive\\Documents\\GitHub\\TPRelex\\RELEX-Y\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "C:\\Users\\DELL\\OneDrive\\Documents\\GitHub\\TPRelex\\RELEX-Y\\CSPOMS_DEV_Conn\\server-ca.pem"
}

# Creating a connection URL
url = (
    f"postgresql+psycopg2://{db_params['user']}:{urllib.parse.quote(db_params['password'])}"
    f"@{db_params['host']}:{db_params['port']}/{db_params['database']}"
    f"?sslmode={db_params['sslmode']}&sslcert={db_params['sslcert']}&sslkey={db_params['sslkey']}&sslrootcert={db_params['sslrootcert']}"
)
# print(url)
# Creating an SQLAlchemy engine
engine = create_engine(url)

# Define your SQL query
select_query = '''
select 
		poh.po_num as purchase_order_number,
		poh.po_date_ordered as po_creation_date,
		poh.act_ship_to_fac as facility,
        pol.po_header_id,
        pol.pol_line_id as po_line_id,
        pol.ssic as item,
        pol.item_warehouse_code as whr_item,
        pol.order_qty,
        pol.pol_po_selection_fac,
        im.VENDOR_TIE as V_TI_new,
        pol.POL_VENDOR_TI as V_TI_old,
        im.VENDOR_TIER as V_HI_new,
        pol.POL_VENDOR_HI as V_HI_old,
        im.WHSE_TIE as W_TI_new,
        pol.POL_WHSE_TI as W_TI_old,
        im.WHSE_TIER as W_HI_new,
        pol.POL_WHSE_HI as W_HI_old,
        poh.RECEIVED_DATE,
        pol.PALLET_QUANTITY as OLD_PALLET_QUANTITY,
        pol.VEN_PALLET_QTY as OLD_VEN_PALLET_QTY,
        -- Calculating PALLET_QUANTITY and VEN_PALLET_QTY
		round(pol.order_qty / (im.VENDOR_TIE * im.VENDOR_TIER),2) AS PALLET_QUANTITY,
		round(pol.order_qty / (im.WHSE_TIE * im.WHSE_TIER),2) AS VEN_PALLET_QTY
 
    FROM 
        XXPO_PO_HEADERS poh
    JOIN 
        XXPO_PO_LINES pol ON pol.po_header_id = poh.po_header_id
    JOIN 
        XXPO_ITEM_MASTER im ON LPAD(pol.ssic, 10, '0') = im.item_ssic
    WHERE 
        poh.RECEIVED_DATE IS NOT NULL
        AND 
        poh.PO_STATUS IN ('Open', 'Pre-Approved')
        AND im.facility_code = pol.pol_po_selection_fac
        AND (
            CASE 
                WHEN pol.POL_VENDOR_TI = im.VENDOR_TIE THEN 'Y'
                ELSE 'N'
            END = 'N'
            OR
            CASE 
                WHEN pol.POL_VENDOR_HI = im.VENDOR_TIER THEN 'Y'
                ELSE 'N'
            END = 'N'
            OR
            CASE 
                WHEN pol.POL_WHSE_TI = im.WHSE_TIE THEN 'Y'
                ELSE 'N'
            END = 'N'
            OR
            CASE 
                WHEN pol.POL_WHSE_HI = im.WHSE_TIER THEN 'Y'
                ELSE 'N'
            END = 'N'
        )
'''

# Execute the query and read the result into a DataFrame using the SQLAlchemy engine
df = pd.read_sql_query(select_query, engine)
# Convert NaN values to None
df = df.replace({np.nan: None})

# Display the DataFrame
print(len(df))
print(df)
# Convert all float columns with NaN to int, keeping NaNs as None



2
  purchase_order_number po_creation_date facility  po_header_id  po_line_id  \
0                 26689       2024-08-20      072         18491      179157   
1                 26689       2024-08-20      072         18491      179156   

         item whr_item  order_qty pol_po_selection_fac  v_ti_new  ...  \
0  0000828752    10220      300.0                  072      10.0  ...   
1  0000464191   547503      100.0                  072      10.0  ...   

   v_hi_old  w_ti_new  w_ti_old  w_hi_new  w_hi_old  received_date  \
0      10.0      10.0      10.0      10.0      10.0     2024-10-07   
1      10.0      10.0      10.0      10.0      10.0     2024-10-07   

   old_pallet_quantity old_ven_pallet_qty pallet_quantity ven_pallet_qty  
0                 None               None             3.0            3.0  
1                 None               None             1.0            1.0  

[2 rows x 22 columns]


In [8]:
# df[['pol_vendor_ti','vendor_ti','vendor_ti_check','item_ssic']]
filtered_df = df
len(filtered_df)
filtered_df

,po_header_id,pol_line_id,order_qty,pol_po_selection_fac,vendor_tie,pol_vendor_ti,vendor_tier,pol_vendor_hi,whse_tie,pol_whse_ti,whse_tier,pol_whse_hi,received_date,pallet_quantity,ven_pallet_qty
0,18491,179157,300.0,072,10.0,9.0,10.0,10.0,10.0,10.0,10.0,10.0,2024-10-07,3.0,3.0
1,18491,179156,100.0,072,10.0,11.0,10.0,10.0,10.0,10.0,10.0,10.0,2024-10-07,1.0,1.0


In [14]:
# df = pd.read_sql_query(select_query, engine)

if df.empty:
    print('No changes to update')
else:
    print('update_audit_table()')

update_audit_table()


In [39]:
filtered_vendor_ti_changes = df[df['vendor_ti_check'] == 'N']
filtered_vendor_ti_changes2 = df[df['pol_vendor_ti'] != df['vendor_ti']]
# [['pol_vendor_ti', 'vendor_ti', 'vendor_ti_check', 'item_ssic']]
# filtered_df['audit_date'] = SYSDATE do this in query
# filtered_df['audit_level'] = 'XXPO_PO_LINES'
# filtered_df['user_name'] = 'Nightly Program -E1155'
filtered_vendor_ti_changes
filtered_vendor_ti_changes 
filtered_vendor_ti_changes


,po_header_id,pol_line_id,po_num,po_date_ordered,act_ship_to_fac,pol_vendor_ti,vendor_ti,vendor_ti_check,pol_vendor_hi,vendor_hi,...,whse_ti,whse_ti_check,pol_whse_hi,whse_hi,whse_hi_check,ssic,item_warehouse_code,item_ssic,po_status,received_date
1,102,524,132404,None,303,None,22.0,N,None,5.0,...,22.0,N,None,3.0,N,26375,10,0000026375,Open,2024-08-20
2,102,527,132404,None,303,None,10.0,N,None,7.0,...,10.0,N,None,7.0,N,29771,10,0000029771,Open,2024-08-20
3,102,530,132404,None,303,None,12.0,N,None,8.0,...,12.0,N,None,8.0,N,29831,10,0000029831,Open,2024-08-20
4,102,531,132404,None,303,None,12.0,N,None,8.0,...,12.0,N,None,8.0,N,29837,10,0000029837,Open,2024-08-20
5,102,531,132404,None,303,None,12.0,N,None,8.0,...,12.0,N,None,6.0,N,29837,10,0000029837,Open,2024-08-20
7,102,501,132404,None,303,None,15.0,N,None,12.0,...,15.0,N,None,9.0,N,13222,10,0000013222,Open,2024-08-20
8,102,507,132404,None,303,None,15.0,N,None,6.0,...,15.0,N,None,4.0,N,194085,10,0000194085,Open,2024-08-20
9,102,507,132404,None,303,None,15.0,N,None,6.0,...,15.0,N,None,5.0,N,194085,10,0000194085,Open,2024-08-20
10,102,508,132404,None,303,None,13.0,N,None,6.0,...,13.0,N,None,6.0,N,201018,10,0000201018,Open,2024-08-20
11,102,508,132404,None,303,None,13.0,N,None,6.0,...,13.0,N,None,5.0,N,201018,10,0000201018,Open,2024-08-20


In [ ]:
def insert_data_into_postgres(self,data_df, cursor, connection, table_name):
    try:
        # Handling numeric columns
        # is_numeric_columns = data_df.apply(lambda col: pd.to_numeric(col, errors='coerce').notna().all())
        # transformed_rows = []
        insert_query = f"INSERT INTO {table_name} ({', '.join(data_df.columns)}) VALUES ({', '.join(['%s']*len(data_df.columns))})"
        data_df = data_df.fillna('')
        data_df = data_df.astype(str)
        data_df = data_df.replace('',None)
        tuples = data_df.values.tolist()
        try:
            cursor.executemany(insert_query,tuples)
            connection.commit()
            print(f"{data_df.shape[0]} rows are loaded to table")
            return data_df.shape[0]
            # cursor.close()
            # connection.close()

        except Exception as error:
            print("Error: %s" % error)
            connection.rollback()
            cursor.close()

    except Exception as e:
        # Handle other unexpected exceptions
        print(f"An unexpected error occurred: {e}")

In [30]:
import pandas as pd
from datetime import datetime

def update_audit_table(df,Column_name, new_column_value, old_column_value, request_id,cursor,connection):
    # Compare values in the two columns and filter rows with mismatched values
    mismatched_df = df[df[new_column_value] != df[old_column_value]]
    if not mismatched_df.empty:
        # Prepare the data for audit insertion
#         mismatched_df['audit_date'] = datetime.now().strftime('%Y-%m-%d')  # Adding current date as audit_date
        # Assuming df is your DataFrame and new_column_value, old_column_value are variables holding the column names
        mismatched_df = mismatched_df.rename(columns={new_column_value: 'new_value', old_column_value: 'old_value'})
        mismatched_df['user_name'] = 'Nighty Program'  # Adding static user_name
        mismatched_df['attribute_change'] = Column_name # Column name indicating the change
        mismatched_df['request_id'] = request_id
        mismatched_df['audit_level'] = 'LINES'


        # Selecting only the relevant columns to insert into the audit table
        audit_columns = [
            'audit_level',
            'purchase_order_number', 
            'po_creation_date', 
            'facility', 
            'po_header_id', 
            'po_line_id', 
            'user_name', 
            'attribute_change',
            'item',
            'whr_item',
            'old_value',
            'new_value',
            'request_id'
        ]

        # Subsetting the DataFrame with the required columns
        data_df = mismatched_df[audit_columns]

        # Insert the data into xxpo_po_audit table
        insert_query = f"INSERT INTO xxpo_po_audit (audit_date,creation_date,last_update_date,created_by,last_updated_by,process_status,{', '.join(data_df.columns)}) VALUES (current_date,current_timestamp,current_timestamp,'CSPOMS','CSPOMS','P',{', '.join(['%s']*len(data_df.columns))})"
        data_df = data_df.fillna('')
        data_df = data_df.astype(str)
        data_df = data_df.replace('',None)
        tuples = data_df.values.tolist()
        try:
            print(insert_query,tuples)
            cursor.executemany(insert_query,tuples)
            connection.commit()
            print(f"{data_df.shape[0]} rows are loaded to table")
            return data_df.shape[0]
            cursor.close()
            connection.close()

        except Exception as error:
            print("Error: %s" % error)
            connection.rollback()
            cursor.close()

    else:
        print('No mismatches found, nothing to update.')

# Example usage
# Assuming df, engine are defined, and the columns to compare are 'column1' and 'column2'
update_audit_table(df,'VENDOR_TI', 'v_ti_new', 'v_ti_old',777,cursor,connection)
update_audit_table(df,'VENDOR_HI', 'v_hi_new', 'v_hi_old',777,cursor,connection)
update_audit_table(df,'WHSE_TI', 'w_ti_new', 'w_ti_old',777,cursor,connection)
update_audit_table(df,'WHSE_HI', 'w_hi_new', 'w_hi_old',777,cursor,connection)
update_audit_table(df,'PALLET_QUANTITY', 'pallet_quantity', 'old_pallet_quantity',777,cursor,connection)
update_audit_table(df,'VEN_PALLET_QTY', 'ven_pallet_qty', 'old_ven_pallet_qty',777,cursor,connection)




INSERT INTO xxpo_po_audit (audit_date,creation_date,last_update_date,created_by,last_updated_by,process_status,audit_level, purchase_order_number, po_creation_date, facility, po_header_id, po_line_id, user_name, attribute_change, item, whr_item, old_value, new_value, request_id) VALUES (current_date,current_timestamp,current_timestamp,'CSPOMS','CSPOMS','P',%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) [['LINES', '26689', '2024-08-20', '072', '18491', '179157', 'Nighty Program', 'VENDOR_TI', '0000828752', '10220', '9.0', '10.0', '777'], ['LINES', '26689', '2024-08-20', '072', '18491', '179156', 'Nighty Program', 'VENDOR_TI', '0000464191', '547503', '11.0', '10.0', '777']]
2 rows are loaded to table
No mismatches found, nothing to update.
No mismatches found, nothing to update.
No mismatches found, nothing to update.
INSERT INTO xxpo_po_audit (audit_date,creation_date,last_update_date,created_by,last_updated_by,process_status,audit_level, purchase_order_number, po_creation_date, fa

2

In [37]:
def update_audit_table(df, audit_level, Column_name, new_column_value, old_column_value, request_id, cursor, connection):
    # Compare values in the two columns and filter rows with mismatched values
    mismatched_df = df[df[new_column_value] != df[old_column_value]]
    
    if not mismatched_df.empty:
        # Prepare the data for audit insertion
        mismatched_df = mismatched_df.rename(columns={new_column_value: 'new_value', old_column_value: 'old_value'})
        mismatched_df['user_name'] = 'Nighty Program'  # Adding static user_name
        mismatched_df['attribute_change'] = Column_name  # Column name indicating the change
        mismatched_df['request_id'] = request_id
        mismatched_df['audit_level'] = audit_level  # Set the audit level (HEADERS or LINES)

        # Define common columns for both HEADERS and LINES
        audit_columns = [
            'audit_level',
            'purchase_order_number', 
            'po_creation_date', 
            'facility', 
            'po_header_id', 
            'user_name', 
            'attribute_change',
            'old_value',
            'new_value',
            'request_id'
        ]

        # Extend with LINES-specific columns if the audit level is 'LINES'
        if audit_level == 'LINES':
            audit_columns += ['po_line_id', 'item', 'whr_item']

        # Subsetting the DataFrame with the required columns
        data_df = mismatched_df[audit_columns]

        # Insert the data into xxpo_po_audit table
        insert_query = f"INSERT INTO xxpo_po_audit (audit_date,creation_date,last_update_date,created_by,last_updated_by,process_status,{', '.join(data_df.columns)}) VALUES (current_date,current_timestamp,current_timestamp,'CSPOMS','CSPOMS','P',{', '.join(['%s']*len(data_df.columns))})"
        data_df = data_df.fillna('')  # Replace NaN values with empty strings
        data_df = data_df.astype(str)  # Ensure all data types are strings
        data_df = data_df.replace('', None)  # Replace empty strings with None (for SQL NULL)
        tuples = data_df.values.tolist()  # Convert the DataFrame into a list of tuples

        try:
            print(insert_query, tuples)  # For debugging purposes
            cursor.executemany(insert_query, tuples)  # Execute the insertion query
            connection.commit()  # Commit the transaction
            print(f"{data_df.shape[0]} rows are loaded to the table")
            return data_df.shape[0]  # Return the number of rows inserted
            cursor.close()
            connection.close()

        except Exception as error:
            print("Error: %s" % error)  # Print the error if it occurs
            connection.rollback()  # Rollback the transaction in case of error
            cursor.close()

    else:
        print('No mismatches found, nothing to update.')

        
        # Assuming df, engine are defined, and the columns to compare are 'column1' and 'column2'
update_audit_table(df,'LINES','VENDOR_TI', 'v_ti_new', 'v_ti_old',777,cursor,connection)
update_audit_table(df,'LINES','VENDOR_HI', 'v_hi_new', 'v_hi_old',777,cursor,connection)
update_audit_table(df,'LINES','WHSE_TI', 'w_ti_new', 'w_ti_old',777,cursor,connection)
update_audit_table(df,'LINES','WHSE_HI', 'w_hi_new', 'w_hi_old',777,cursor,connection)
update_audit_table(df,'LINES','PALLET_QUANTITY', 'pallet_quantity', 'old_pallet_quantity',777,cursor,connection)
update_audit_table(df,'LINES','VEN_PALLET_QTY', 'ven_pallet_qty', 'old_ven_pallet_qty',777,cursor,connection)
update_audit_table(header_df, 'HEADERS', 'PO_WHSE_PALLET_QUANTITY', 'new_po_whse_pallet_quantity', 'old_po_whse_pallet_quantity', 777, cursor, connection)
update_audit_table(header_df, 'HEADERS', 'PO_VNDR_PALLET_QUANTITY', 'new_po_vndr_pallet_quantity', 'old_po_vndr_pallet_quantity', 777, cursor, connection)


INSERT INTO xxpo_po_audit (audit_date,creation_date,last_update_date,created_by,last_updated_by,process_status,audit_level, purchase_order_number, po_creation_date, facility, po_header_id, user_name, attribute_change, old_value, new_value, request_id, po_line_id, item, whr_item) VALUES (current_date,current_timestamp,current_timestamp,'CSPOMS','CSPOMS','P',%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) [['LINES', '26689', '2024-08-20', '072', '18491', 'Nighty Program', 'VENDOR_TI', '9.0', '10.0', '777', '179157', '0000828752', '10220'], ['LINES', '26689', '2024-08-20', '072', '18491', 'Nighty Program', 'VENDOR_TI', '11.0', '10.0', '777', '179156', '0000464191', '547503']]
2 rows are loaded to the table
No mismatches found, nothing to update.
No mismatches found, nothing to update.
No mismatches found, nothing to update.
INSERT INTO xxpo_po_audit (audit_date,creation_date,last_update_date,created_by,last_updated_by,process_status,audit_level, purchase_order_number, po_creation_date

1

In [36]:
header_query = '''
-- CTE to find po_header_id with TI/HI differences
WITH header_with_differences AS (
    SELECT DISTINCT
        pol.po_header_id
    FROM 
        XXPO_PO_LINES pol
    JOIN 
        XXPO_ITEM_MASTER im ON LPAD(pol.ssic, 10, '0') = im.item_ssic
    WHERE 
        pol.po_header_id IN (
            SELECT po_header_id
            FROM XXPO_PO_HEADERS poh
            WHERE poh.RECEIVED_DATE IS NOT NULL
            AND poh.PO_STATUS IN ('Open', 'Pre-Approved')
        )
        AND im.facility_code = pol.pol_po_selection_fac
        -- Check for TI/HI differences
        AND (
                pol.POL_VENDOR_TI <> im.VENDOR_TIE 
            OR
                pol.POL_VENDOR_HI <> im.VENDOR_TIER 
            OR
                pol.POL_WHSE_TI <> im.WHSE_TIE 
            OR
                pol.POL_WHSE_HI <> im.WHSE_TIER 
        )
),

-- CTE to calculate new pallet quantities for each po_header_id
pallet_sums AS (
    SELECT
        pol.po_header_id,
        SUM(ROUND(pol.order_qty / (im.WHSE_TIE * im.WHSE_TIER), 2)) AS new_PO_WHSE_PALLET_QUANTITY,
        SUM(ROUND(pol.order_qty / (im.VENDOR_TIE * im.VENDOR_TIER), 2)) AS new_PO_VNDR_PALLET_QUANTITY
    FROM 
        XXPO_PO_LINES pol
    JOIN 
        XXPO_ITEM_MASTER im ON LPAD(pol.ssic, 10, '0') = im.item_ssic
    WHERE 
        pol.po_header_id IN (SELECT po_header_id FROM header_with_differences)
        AND im.facility_code = pol.pol_po_selection_fac
    GROUP BY 
        pol.po_header_id
)

-- Select old and new values for the po_header_id that are being updated
SELECT 
    poh.po_header_id,
    poh.po_num as purchase_order_number,
	poh.po_date_ordered as po_creation_date,
	poh.act_ship_to_fac as facility,
	poh.RECEIVED_DATE,
    poh.po_status,
    poh.PO_WHSE_PALLET_QUANTITY AS old_PO_WHSE_PALLET_QUANTITY,
    ROUND(ps.new_PO_WHSE_PALLET_QUANTITY) as new_PO_WHSE_PALLET_QUANTITY,
    poh.PO_VNDR_PALLET_QUANTITY AS old_PO_VNDR_PALLET_QUANTITY,
    ROUND(ps.new_PO_VNDR_PALLET_QUANTITY) as new_PO_VNDR_PALLET_QUANTITY
FROM 
    XXPO_PO_HEADERS poh
JOIN 
    pallet_sums ps ON poh.po_header_id = ps.po_header_id
WHERE 
    poh.PO_STATUS IN ('Open', 'Pre-Approved')

'''
# post_cur.execute(select_query)
header_df = pd.read_sql_query(header_query,connection)
header_df

C:\Users\DELL\AppData\Local\Temp\ipykernel_65748\3201081515.py:68: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  header_df = pd.read_sql_query(header_query,connection)


,po_header_id,purchase_order_number,po_creation_date,facility,received_date,po_status,old_po_whse_pallet_quantity,new_po_whse_pallet_quantity,old_po_vndr_pallet_quantity,new_po_vndr_pallet_quantity
0,18622,26820,2024-09-03,001,2024-10-08,Open,None,56.0,None,42.0


In [19]:
df

,purchase_order_number,po_creation_date,facility,po_header_id,po_line_id,item,whr_item,order_qty,pol_po_selection_fac,v_ti_new,v_ti_old,v_hi_new,v_hi_old,w_ti_new,w_ti_old,w_hi_new,w_hi_old,received_date,pallet_quantity,ven_pallet_qty
0,26689,2024-08-20,072,18491,179157,0000828752,10220,300.0,072,10.0,9.0,10.0,10.0,10.0,10.0,10.0,10.0,2024-10-07,3.0,3.0
1,26689,2024-08-20,072,18491,179156,0000464191,547503,100.0,072,10.0,11.0,10.0,10.0,10.0,10.0,10.0,10.0,2024-10-07,1.0,1.0
